# BIOMAG introduction material 2. - Pytorch basics

In [1]:
import torch # pytorch basic package
from torch import nn # neural net 
from torch.utils.data import DataLoader, Dataset # to work with data
from torchvision import datasets # built-in data
from torchvision.transforms import ToTensor # to convert nparrays/images into pytorch tensors

First, let's check if the GPU is available:

In [2]:
torch.cuda.is_available()

True

Now, let us download the datasets that we are going to use:

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



The PyTorch ``DataLoader`` acts as a wrapper over the ``Dataset`` and supports batching, random sampling, augmentation, etc., basically everything you would need for your neural network research.

In [7]:
batch_size = 32

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

print(test_data[1][0].shape)

Shape of X [N, C, H, W]: torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]) torch.int64
torch.Size([1, 28, 28])
